In [1]:
import pandas as pd
import numpy as np
import requests
import shutil
import tempfile
import zipfile
from pathlib import Path
from typing import Dict, List

from sklearn import preprocessing
from sklearn import utils

from sklearn.model_selection import GridSearchCV, KFold, train_test_split, ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler #Para estandarizar los datos

from sklearn.tree import DecisionTreeClassifier # árbol de decisión
from sklearn.naive_bayes import GaussianNB # bayes
from sklearn.neural_network import MLPClassifier #import para la red neuronal
from sklearn.svm import SVC #import para svm

#TODO
#Agruegue más "import" de ser necesario

In [24]:
def extract(url: str) -> Path:
    """Obtener la data desde el origen. La data está comprimida. El resultado queda
    alojado en un directorio temporal

    Parameters
    ----------
    url : str
        La URI del dataset.

    Returns
    -------
    Path
        La ruta del directorio temporal donde los datos son almacenados.
    """
    tmp_folder = Path(tempfile.mkdtemp())
    print(tmp_folder, "este es el folder temporal que se crea")
    local_filename = tmp_folder / url.split('/')[-1]
    # download file
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
   # data.zip -> temp_file
   # unzip
    # TODO Descomente esta parte si es necesario descomprimir
    with zipfile.ZipFile(local_filename, "r") as zip_ref:
        zip_ref.extractall(tmp_folder)
    return tmp_folder

def transform(folder: Path)->Dict:
    """ Transformar el dataset para que sea leible fácilmente dentro del sandbox.

    Parameters
    ----------
    folder : Path
        La ruta del directorion donde los datos fueron almacenados

    Returns
    -------
    Dict
        Datos transformados
    """
    # leer datos y transformar imagenes a vectores
    #TODO: Modificar según sea su dataset 
    #                   
    input_file = "./datalab2.csv" 
    columns = [  #'sequence Name',
        'mcg',
        'gvh',
        'lip',
        'chg',
        'aac',
        'alm1',
        'alm2',
        'class distribution'
    ]
    class_names = [
        'cp',
        'im',
        'imS',
        'imL',
        'imU',
        'omL',
        'om',
        'pp',     
    ]

    df = pd.read_csv(input_file,delimiter=";",)
    print(df)
    return df, columns, class_names

def classify(X:pd.DataFrame, y:pd.DataFrame, class_names:List[str], estimator):
    print("estos son los valores que entran : x,y,class_names,stimador ",X,y,class_names)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    grid = GridSearchCV(
        estimator=estimator,
        param_grid={},
        n_jobs=-1,
        verbose=2,
        cv=KFold(
            n_splits=5, shuffle=True, random_state=10
        ),
    )
    print ("valores de xtrain y y : ",X_train,y_train)
    grid.fit(X_train, y=y_train)

    y_pred = grid.best_estimator_.predict(X_test)

    print(
        classification_report(
            y_test,
            y_pred,
            target_names=class_names,
            zero_division=0,
        )
    )

In [16]:
#TODO:Modificar la url
folder = extract("https://archive.ics.uci.edu/static/public/39/ecoli.zip")


/tmp/tmp4f4f_xoe este es el folder temporal que se crea


In [25]:
df, columns, class_names = transform(folder)

    sequence Name   mcg   gvh   lip  chg   aac  alm1  alm2 class distribution
0       AAT_ECOLI  0.49  0.29  0.48  0.5  0.56  0.24  0.35                 cp
1      ACEA_ECOLI  0.07  0.40  0.48  0.5  0.54  0.35  0.44                 cp
2      ACEK_ECOLI  0.56  0.40  0.48  0.5  0.49  0.37  0.46                 cp
3      ACKA_ECOLI  0.59  0.49  0.48  0.5  0.52  0.45  0.36                 cp
4       ADI_ECOLI  0.23  0.32  0.48  0.5  0.55  0.25  0.35                 cp
..            ...   ...   ...   ...  ...   ...   ...   ...                ...
331    TREA_ECOLI  0.74  0.56  0.48  0.5  0.47  0.68  0.30                 pp
332    UGPB_ECOLI  0.71  0.57  0.48  0.5  0.48  0.35  0.32                 pp
333    USHA_ECOLI  0.61  0.60  0.48  0.5  0.44  0.39  0.38                 pp
334    XYLF_ECOLI  0.59  0.61  0.48  0.5  0.42  0.42  0.37                 pp
335    YTFQ_ECOLI  0.74  0.74  0.48  0.5  0.31  0.53  0.52                 pp

[336 rows x 9 columns]


In [26]:
X = df[columns[:-1]]
print(X)
y = df[columns[-1]]
print(y)
print("valores de x, y antes de classify ",X,y)

      mcg   gvh   lip  chg   aac  alm1  alm2
0    0.49  0.29  0.48  0.5  0.56  0.24  0.35
1    0.07  0.40  0.48  0.5  0.54  0.35  0.44
2    0.56  0.40  0.48  0.5  0.49  0.37  0.46
3    0.59  0.49  0.48  0.5  0.52  0.45  0.36
4    0.23  0.32  0.48  0.5  0.55  0.25  0.35
..    ...   ...   ...  ...   ...   ...   ...
331  0.74  0.56  0.48  0.5  0.47  0.68  0.30
332  0.71  0.57  0.48  0.5  0.48  0.35  0.32
333  0.61  0.60  0.48  0.5  0.44  0.39  0.38
334  0.59  0.61  0.48  0.5  0.42  0.42  0.37
335  0.74  0.74  0.48  0.5  0.31  0.53  0.52

[336 rows x 7 columns]
0      cp
1      cp
2      cp
3      cp
4      cp
       ..
331    pp
332    pp
333    pp
334    pp
335    pp
Name: class distribution, Length: 336, dtype: object
valores de x, y antes de classify        mcg   gvh   lip  chg   aac  alm1  alm2
0    0.49  0.29  0.48  0.5  0.56  0.24  0.35
1    0.07  0.40  0.48  0.5  0.54  0.35  0.44
2    0.56  0.40  0.48  0.5  0.49  0.37  0.46
3    0.59  0.49  0.48  0.5  0.52  0.45  0.36
4    0.23  0.

In [27]:
# TODO: árbol de decisión
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)
dt_classifier = DecisionTreeClassifier(criterion="gini")
classify(X, y_transformed, class_names, dt_classifier)


estos son los valores que entran : x,y,class_names,stimador        mcg   gvh   lip  chg   aac  alm1  alm2
0    0.49  0.29  0.48  0.5  0.56  0.24  0.35
1    0.07  0.40  0.48  0.5  0.54  0.35  0.44
2    0.56  0.40  0.48  0.5  0.49  0.37  0.46
3    0.59  0.49  0.48  0.5  0.52  0.45  0.36
4    0.23  0.32  0.48  0.5  0.55  0.25  0.35
..    ...   ...   ...  ...   ...   ...   ...
331  0.74  0.56  0.48  0.5  0.47  0.68  0.30
332  0.71  0.57  0.48  0.5  0.48  0.35  0.32
333  0.61  0.60  0.48  0.5  0.44  0.39  0.38
334  0.59  0.61  0.48  0.5  0.42  0.42  0.37
335  0.74  0.74  0.48  0.5  0.31  0.53  0.52

[336 rows x 7 columns] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


In [29]:
# TODO: SVM
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)
svm_classifier = SVC()
classify(X, y_transformed, class_names, svm_classifier)


estos son los valores que entran : x,y,class_names,stimador        mcg   gvh   lip  chg   aac  alm1  alm2
0    0.49  0.29  0.48  0.5  0.56  0.24  0.35
1    0.07  0.40  0.48  0.5  0.54  0.35  0.44
2    0.56  0.40  0.48  0.5  0.49  0.37  0.46
3    0.59  0.49  0.48  0.5  0.52  0.45  0.36
4    0.23  0.32  0.48  0.5  0.55  0.25  0.35
..    ...   ...   ...  ...   ...   ...   ...
331  0.74  0.56  0.48  0.5  0.47  0.68  0.30
332  0.71  0.57  0.48  0.5  0.48  0.35  0.32
333  0.61  0.60  0.48  0.5  0.44  0.39  0.38
334  0.59  0.61  0.48  0.5  0.42  0.42  0.37
335  0.74  0.74  0.48  0.5  0.31  0.53  0.52

[336 rows x 7 columns] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
